In [2]:
import sagemaker
import boto3
import json

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()
region = sess.boto_region_name
prefix = 'sagemaker-online-transform'

In [3]:
import numpy as np
import os
from tensorflow.python.keras.datasets import imdb

In [4]:
csv_test_dir_prefix = 'imdb_data/test'
csv_test_filename = 'test.csv'

x_test = np.loadtxt(f'{csv_test_dir_prefix}/{csv_test_filename}', delimiter=',', dtype='int')

print('x_test shape:', x_test.shape)


x_test shape: (25000, 400)


In [5]:
max_features = 20000
maxlen = 400

_ , (_, y_test) = imdb.load_data(num_words=max_features)


In [6]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow.attach('imdb-tf-2022-04-16-18-40-44')
estimator


2022-04-16 18:58:22 Starting - Preparing the instances for training
2022-04-16 18:58:22 Downloading - Downloading input data
2022-04-16 18:58:22 Training - Training image download completed. Training in progress.
2022-04-16 18:58:22 Uploading - Uploading generated training model
2022-04-16 18:58:22 Completed - Training job completed


In [8]:
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type = 'ml.c5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

In [9]:
predictor.endpoint_name

'imdb-tf-2022-04-16-18-40-44-2022-04-17-13-08-39-209'

In [10]:
data_index = 199

In [12]:
prediction = predictor.predict(x_test[data_index])    
prediction

{'predictions': [[2.0325507e-07]]}

In [13]:
def get_sentiment(score):
    return 'positive' if score > 0.5 else 'negative' 

import re

regex = re.compile(r'^[\?\s]+')
word_index = imdb.get_word_index()

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
first_decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') 
                                 for i in x_test[data_index]])
regex.sub('', first_decoded_review)

"i watched this movie purely for the setting it was filmed in an old hotel that a friend owns shares of the plot was predictable the acting was ? at best the scares were all gross outs not true scares br br i don't remember much of the plot and i think that's because there wasn't much of one to remember they didn't even use the hotel to it's fullest potential the beaches are fantastic and the hotel is situated on a ? at low tide you can walk almost 1 4 mile into the bay which is actually an eerie sight first thing in the morning or late at night when the wind is howling through the cracks br br the best way to see this movie is with the remote in your hand so you can fast forward through the action and i'm using that term ? scenes and pause at the beauty of the surroundings"

In [14]:
print(f'Labeled sentiment for this review is {get_sentiment(y_test[data_index])}')
print(f'Predicted sentiment is {get_sentiment(prediction["predictions"][0][0])}')

Labeled sentiment for this review is negative
Predicted sentiment is negative


In [15]:
predictor.predict(x_test[:5000]) # this would throw an error due to large volume

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from primary with message "<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0</center>
</body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/imdb-tf-2022-04-16-18-40-44-2022-04-17-13-08-39-209 in account 104877823522 for more information.

In [21]:
sagemaker_client = sess.boto_session.client('sagemaker')
autoscaling_client = sess.boto_session.client('application-autoscaling')

endpoint_name = predictor.endpoint_name
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
print(response)

{'EndpointName': 'imdb-tf-2022-04-16-18-40-44-2022-04-17-13-08-39-209', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:104877823522:endpoint/imdb-tf-2022-04-16-18-40-44-2022-04-17-13-08-39-209', 'EndpointConfigName': 'imdb-tf-2022-04-16-18-40-44-2022-04-17-13-08-39-209', 'ProductionVariants': [{'VariantName': 'AllTraffic', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.1-cpu', 'ResolvedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference@sha256:0803dbfa124acf1d43f1493f84d55a435e5894bbdd839fdefe20c750e815bca0', 'ResolutionTime': datetime.datetime(2022, 4, 17, 13, 8, 40, 585000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2022, 4, 17, 13, 8, 39, 878000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2022, 4, 17, 13, 10, 33, 382000, tzinfo=tzlocal()), 'ResponseM

In [25]:
# application autoscaling references the endpoint using string below

resource_id = f'endpoint/{endpoint_name}/variant/AllTraffic'
response = autoscaling_client.register_scalable_target(
    ServiceNamespace = 'sagemaker',
    ResourceId = resource_id,
    ScalableDimension = 'sagemaker:variant:DesiredInstanceCount',
    MinCapacity = 1,
    MaxCapacity = 4
)

response = autoscaling_client.put_scaling_policy(
    PolicyName = 'Invocations-ScalingPolicy',
    ServiceNamespace = 'sagemaker',
    ResourceId = resource_id,
    ScalableDimension = 'sagemaker:variant:DesiredInstanceCount',
    PolicyType = 'TargetTrackingScaling', # other options are StepScaling, 
    TargetTrackingScalingPolicyConfiguration = {
        'TargetValue': 4000.0,
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance', 
        },
        'ScaleInCooldown': 600, 
        'ScaleOutCooldown': 300,
        'DisableScaleIn': False # If true, scale-in is disabled.
    }
)

In [26]:
response = autoscaling_client.describe_scaling_policies(
    ServiceNamespace='sagemaker'
)

for i in response['ScalingPolicies']:
    print('')
    print(i['PolicyName'])
    print('')
    if('TargetTrackingScalingPolicyConfiguration' in i):
        print(i['TargetTrackingScalingPolicyConfiguration']) 
    else:
        print(i['StepScalingPolicyConfiguration'])
    print('')


Invocations-ScalingPolicy

{'TargetValue': 4000.0, 'PredefinedMetricSpecification': {'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'}, 'ScaleOutCooldown': 300, 'ScaleInCooldown': 600, 'DisableScaleIn': False}

